# Data Streaming 2: Connecting to Tick Data Server & Generating a Signal

# Contents

## 1. Running a Tick Data Server
    - Generate data with geometric brownian motion
    - Create a socket using ZeroMQ and publish data to socket

## 2. Connecting a Tick Data Client

## 3. Real Time Signal Generation 

## 4. Plotting Streaming Data

In [1]:
import zmq
import math
import time
import random
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 2. Connecting a Tick Data Client

In [2]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [ ]:
"""
while True:
    data = socket.recv_string() 
    print(data, end='\r')
    
"""

# 3. Real Time Signal Generation 

In [3]:
df = pd.DataFrame() 
mom=3
min_length = mom + 1

while True:
    data = socket.recv_string()
    t = datetime.datetime.now()
    sym, value = data.split()
    df = df.append(pd.DataFrame({sym: float(value)}, index=[t]))
    #print("df = ", df[sym])
    
    dr = df.resample('5s', label='right').last() 
    #print("dr = ", dr[sym][-1])
    #print("inside log = ", dr[sym][-1]/dr[sym].shift(1)[-1])
    #print("log return = ", np.log(dr[sym][-1]/dr[sym].shift(1)[-1]))
    
    dr['returns'] = np.log(dr[sym][-1]/dr[sym].shift(1)[-1])
    if len(dr) > min_length:
        min_length += 1
        dr['momentum'] = np.sign(dr['returns'].rolling(mom).mean()) 
        print('\n' + '=' * 51)
        print('NEW SIGNAL | {}'.format(datetime.datetime.now())) 
        print('=' * 51)
        print(dr.iloc[:-1].tail())
        if dr['momentum'].iloc[-2] == 1.0:
            print('\nLong market position.')
            # take some action (e.g., place buy order)
        elif dr['momentum'].iloc[-2] == -1.0: 
            print('\nShort market position.')
            # take some action (e.g., place sell order)


NEW SIGNAL | 2023-03-23 22:41:10.809951
                     SYMBOL   returns  momentum
2023-03-23 22:40:55  121.36 -0.003294       NaN
2023-03-23 22:41:00  122.24 -0.003294       NaN
2023-03-23 22:41:05  121.06 -0.003294      -1.0
2023-03-23 22:41:10  121.64 -0.003294      -1.0

Short market position.

NEW SIGNAL | 2023-03-23 22:41:15.956676
                     SYMBOL   returns  momentum
2023-03-23 22:40:55  121.36 -0.005158       NaN
2023-03-23 22:41:00  122.24 -0.005158       NaN
2023-03-23 22:41:05  121.06 -0.005158      -1.0
2023-03-23 22:41:10  121.64 -0.005158      -1.0
2023-03-23 22:41:15  120.51 -0.005158      -1.0

Short market position.

NEW SIGNAL | 2023-03-23 22:41:20.412880
                     SYMBOL   returns  momentum
2023-03-23 22:41:00  122.24  0.001752       NaN
2023-03-23 22:41:05  121.06  0.001752       1.0
2023-03-23 22:41:10  121.64  0.001752       1.0
2023-03-23 22:41:15  120.51  0.001752       1.0
2023-03-23 22:41:20  119.79  0.001752       1.0

Long market 

KeyboardInterrupt: 